In [31]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

### MATRIZ BCH

Esta matríz se ha obtenido del artículo científico: `Brightness Calculation in Digital Image Processing` y nos ayudará a calcular el brillo de una imagen.

In [32]:
MATRIZ_toDEF = np.array([[0.2053, 0.7125, 0.4670],
                        [1.8537, -1.2797, -0.4429],
                        [-0.3655, 1.0120, -0.6014]])

print(MATRIZ_toDEF) 

[[ 0.2053  0.7125  0.467 ]
 [ 1.8537 -1.2797 -0.4429]
 [-0.3655  1.012  -0.6014]]


In [33]:
MATRIZ_toXYZ = np.array([[0.6712, 0.4955, 0.1540],
                        [0.7061, 0.0248, 0.5223],
                        [0.7689, -0.2556, -0.8645]])

### 🧮 Cálculo del Brillo utilizando la Métrica de Cohen  

La función `obtener_brillo` calcula el brillo de una imagen a partir de sus componentes en el espacio de color XYZ, utilizando la métrica de Cohen basada en el modelo BCH (Brightness, Chroma, Hue).  

**¿Cómo funciona?**  
- Transformación BCH:  
   - Se toma la imagen en sus componentes **X, Y, Z** (R, G, B) y se aplica la matriz auxiliar para obtener nuevas representaciones **D, E, F**.    

- Cálculo brillo:
   - A partir de los valores **D, E, F**, se calcula el brillo **B** como la norma euclidiana en este espacio:  
     
     ```math
     B = \sqrt{D^2 + E^2 + F^2}
     ```
   - Luego, se obtiene el **brillo promedio**  de toda la imagen.  


In [34]:
def calc_metricas_cohen(X, Y, Z):
    alto, ancho = X.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_xyz = np.stack([X, Y, Z], axis=-1).reshape(-1, 3)
    # Aplicar la matriz DEF
    cohen = pixels_xyz @ MATRIZ_toDEF.T  # Resultado (N, 3)
    return cohen.reshape(alto, ancho, 3)

def calc_bch_to_xyz(B, C, H):
    alto, ancho = B.shape
    # Stack y reshape en una matriz (N, 3)
    pixels_bch = np.stack([B, C, H], axis=-1).reshape(-1, 3)
    # Aplicar la matriz XYZ
    xyz = pixels_bch @ MATRIZ_toXYZ.T  # Resultado (N, 3)
    return xyz.reshape(alto, ancho, 3)

def obtener_brillo_valores_def(X, Y, Z):
    cohen = calc_metricas_cohen(X, Y, Z)
    D, E, F = cohen[:, :, 0], cohen[:, :, 1], cohen[:, :, 2]
    B = np.sqrt(D**2 + E**2 + F**2)
    return B.mean(), D, E, F

### 🧮 Obtener brillo y canal DEF de la imagen

La función `obtener_brillo_imagen` aplica las funciones creadas en la celda anterior y devuelve el valor del brillo.

La función `obtener_canal_def` aplica las funciones creadas en la celda anterior y devuelve el valor del canal DEF.

**¿Cómo funciona?**  
- Separa la imagen en RGB
- Le pasa cada componente a la función obtener brillo
- Devuelve el brillo de la imagen y el canal DEF en las respectivas funciones

In [ ]:
def obtener_brillo_imagen(imagen_rgb):
    imagen_float = imagen_rgb.astype(np.float32) / 255.0

    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
    
    brillo = obtener_brillo_valores_def(X, Y, Z) [0]
    return brillo

def obtener_def_imagen(imagen_rgb):
    imagen_float = imagen_rgb.astype(np.float32) / 255.0

    X, Y, Z = cv2.split(cv2.cvtColor(imagen_float, cv2.COLOR_RGB2XYZ))
    
    D = obtener_brillo_valores_def(X,Y,Z) [1]
    E = obtener_brillo_valores_def(X,Y,Z) [2]
    F = obtener_brillo_valores_def(X,Y,Z) [3]
    return D, E, F

### 🧮 Cargar imagen RGB   

La función `cargar_imagen_rgb` es una función auxiliar que carga una imagen en rgb .  


In [36]:
def cargar_imagen_rgb(ruta):
    img = cv2.imread(ruta)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {ruta}")
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

### 🗂️ Procesamiento masivo de imágenes calculo del brillo

Esta función `procesar_imagenes_carpeta_bch` aplica la métrica de Cohen para calcular el brillo a todas las imágenes `.png` dentro de una estructura de carpetas.

**Qué hace:**
- Recorre subcarpetas dentro de una carpeta principal.
- Carga la imagen en RGB.
- Aplica la métrica de cohen para obtenter el brillo .
- Guarda el valor del brillo obtenido en un archivo `.txt`.

In [37]:
def procesar_imagenes_carpeta_bch(
        carpeta_entrada="images",
        carpeta_salida="images_procesadas"
):
    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)

        # Añadimos la subcarpeta específica para 'bch'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "bch")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
        
        # Creamos un archivo TXT para guardar los brillos
        ruta_txt = os.path.join(carpeta_salida_sub, "info_brillos_bch.txt")

        with open(ruta_txt, "w", encoding="utf-8") as archivo_txt:
            archivo_txt.write("Nivel de Brillo (método BCH)\n")
            archivo_txt.write(f"Carpeta de imágenes: {ruta_subcarpeta}\n\n")
            archivo_txt.write("Valores mayores corresponden a mayor brillo.\n\n")
            
            # Recorremos los archivos dentro de la subcarpeta
            for filename in sorted(os.listdir(ruta_subcarpeta)):
                if filename.lower().endswith(".png"):
                    ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)
                    
                    # Cargar imagen en rgb
                    img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                    
                    # Calcular brillo
                    brillo = obtener_brillo_imagen(img_rgb)
                    
                    # Guardar resultado en el TXT
                    archivo_txt.write(f"{filename} -> brillo: {brillo:.4f}\n")

    print("Procesamiento completado con método BCH.")

In [38]:
procesar_imagenes_carpeta_bch(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas"
)

Procesamiento completado con método BCH.


### 🧮 Modificar brillo de una imagen   

La función `modificar_brillo` permite ajustar el brillo de una imagen.  

**¿Cómo funciona?**  
- Se determina un coeficiente al llamar a la función 
- Le pasa cada componente a la función obtener brillo
- Devuelve el brillo de la imagen

In [39]:
# m= valores en el scrollbar entre 0.5 y 1.5 (modificar si vemos que aumenta/disminuye demasiado)

def modificar_brillo(rgb, m0=1):
    #algoritmo tv_based
    rgb_nuevo = np.array([rgb[0] * m0, rgb[1] * m0, rgb[2] * m0])
    return rgb_nuevo

In [58]:
def calcular_b_promedio(imagen_rgb, ventana=15):
    """
    Calcula el brillo promedio en un vecindario para cada píxel
    
    Args:
        imagen_rgb: Imagen en formato RGB
        ventana: Tamaño de la ventana para el promediado (ventana x ventana)
    
    Returns:
        Matriz con el brillo promedio para cada píxel
    """
    # Obtener componentes D, E, F
    D, E, F = obtener_def_imagen(imagen_rgb)
    
    # Calcular brillo como la norma euclidiana de D, E, F
    brillo = np.sqrt(D**2 + E**2 + F**2)
    
    # Aplicar filtro de promedio con la ventana especificada
    kernel = np.ones((ventana, ventana), np.float32) / (ventana * ventana)
    b_promedio = cv2.filter2D(brillo, -1, kernel)
    
    return b_promedio

In [59]:
def mejora_contraste(imagen_rgb, k, ventana=15):
    """
    Implementa la mejora de contraste preservando las coordenadas cromáticas
    según la fórmula (9) del paper
    """
    # Obtener componentes D, E, F
    D, E, F = obtener_def_imagen(imagen_rgb)
    
    # Calcular brillo como la norma euclidiana de D, E, F
    brillo = np.sqrt(D**2 + E**2 + F**2)
    
    # Obtener canal BCH
    # Evitar divisiones por cero
    epsilon = 1e-10
    C = np.arccos(np.clip(D / (brillo + epsilon), -1.0, 1.0))
    H = np.arccos(np.clip((E / (brillo + epsilon)) / np.sin(C + epsilon), -1.0, 1.0))
    
    # Calcular luminosidad promedio en el vecindario
    b_promedio = calcular_b_promedio(imagen_rgb, ventana)
    
    # Aplicar la fórmula de mejora de contraste exactamente como en el paper (ecuación 9)
    # B'(m,n) = B_Avr(m,n) * (B(m,n)/B_Avr(m,n))^k
    ratio = brillo / (b_promedio + epsilon)
    nuevo_B = b_promedio * np.power(ratio, k)
    
    # Limitar valores al rango válido
    nuevo_B = np.clip(nuevo_B, 0, 100)
    
    # Reemplazar canal DEF manteniendo las coordenadas cromáticas
    nuevo_D = nuevo_B * np.cos(C)
    nuevo_E = nuevo_B * np.sin(C) * np.cos(H)
    nuevo_F = nuevo_B * np.sin(C) * np.sin(H)
    
    # Convertir de BCH a XYZ
    nuevoXYZ = calc_bch_to_xyz(nuevo_D, nuevo_E, nuevo_F)
    
    # Convertir a float32 antes de usar cv2.cvtColor
    nuevoXYZ = nuevoXYZ.astype(np.float32)
    
    # Convertir de vuelta a RGB
    resultado = cv2.cvtColor(nuevoXYZ, cv2.COLOR_XYZ2RGB) * 255
    
    # Retornar como uint8
    return np.clip(resultado, 0, 255).astype(np.uint8)

In [60]:
def procesar_imagenes_contraste(
    carpeta_entrada="imagenes",
    carpeta_salida="imagenes_procesadas",
    valores_k=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
):

    # Crear la carpeta raíz de salida, si no existe
    os.makedirs(carpeta_salida, exist_ok=True)

    # Iterar sobre todas las subcarpetas de carpeta_entrada
    for subcarpeta in sorted(os.listdir(carpeta_entrada)):
        ruta_subcarpeta = os.path.join(carpeta_entrada, subcarpeta)
        
        # Verificamos si es una carpeta
        if not os.path.isdir(ruta_subcarpeta):
            continue
        
        # Crear subcarpeta de salida correspondiente
        carpeta_salida_sub = os.path.join(carpeta_salida, subcarpeta)
        # Añadimos la subcarpeta específica para 'contraste'
        carpeta_salida_sub = os.path.join(carpeta_salida_sub, "contraste")
        os.makedirs(carpeta_salida_sub, exist_ok=True)
                    
        # Recorremos los archivos dentro de la subcarpeta
        for filename in sorted(os.listdir(ruta_subcarpeta)):
            if filename.lower().endswith("sharp.png"):
                ruta_imagen_entrada = os.path.join(ruta_subcarpeta, filename)

                # Cargar imagen
                img_rgb = cargar_imagen_rgb(ruta_imagen_entrada)
                if img_rgb is None:
                    continue
                
                nombre, _ = os.path.splitext(filename)
                nombre_salida = f"{nombre}_original.jpg"
                ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                # Filtro
                for k in valores_k:
                    contraste_mejorado = mejora_contraste(img_rgb, k)
                    
                    nombre_salida = f"{nombre}_k{k:.1f}.jpg"
                    ruta_imagen_salida = os.path.join(carpeta_salida_sub, nombre_salida)

                    # Convertir de RGB a BGR para guardar con OpenCV
                    contraste_mejorado_bgr = cv2.cvtColor(contraste_mejorado, cv2.COLOR_RGB2BGR)
                    cv2.imwrite(ruta_imagen_salida, contraste_mejorado_bgr)

    print("Procesamiento completado con mejora de contraste.")

In [61]:
# Ejecuta el procesamiento
procesar_imagenes_contraste(
    carpeta_entrada="images",
    carpeta_salida="images_procesadas",
    valores_k=[0.5, 1.0, 1.5, 2.0, 2.5, 3.0]
)

Procesamiento completado con mejora de contraste.


In [62]:
#Medir contraste
def calcular_varianza_histograma(img_gray):
    """
    Calcula la varianza en el histograma para saber el contraste de la imagen 
    Retorna:
        - Varianza (dispersión de intensidades).
    """
       
    # Calcular histograma
    hist = cv2.calcHist([img_gray], [0], None, [256], [0, 256]).flatten()
    
    total_pixeles = img_gray.size
    valores = np.arange(256)                                                # Valores posibles de intensidad (0-255)
    media = np.sum(valores * hist) / total_pixeles
    varianza = np.sum(((valores - media) ** 2) * hist) / total_pixeles

    return varianza

In [63]:
#comprobar bajo contraste y mejorar si es necesario
def mejora_contraste_adaptativo(imagen, kk , k_inicial=1.0, incremento_k=0.1, ventana=20): #kk es el umbral de la varianza del hisograma, cuando se supere considera que una imagen tien buen contraste
    """
    Mejora el contraste de una imagen si la varianza de su histograma es menor que kk.
    Incrementa el parámetro k hasta que la varianza supere el umbral kk.
    """
    imagen_procesada = imagen.copy()
    k_actual = k_inicial
    varianza_actual = calcular_varianza_histograma(imagen_procesada)
    
    while varianza_actual < kk:
        imagen_procesada = mejora_contraste(imagen_procesada, k_actual, ventana)
        varianza_actual = calcular_varianza_histograma(imagen_procesada)
        k_actual += incremento_k
        if k_actual > 3.0:  # Limitar el valor de k para evitar bucles infinitos
            break
    
    return imagen_procesada